In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, Concatenate, RepeatVector, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# 1. Load the Data
# -------------------------------
test_path = '/home/test_data.csv'
train_path = '/home/train_data.csv'
cali_path = '/home/cali_data.csv'

test_unc_path  = '/home/test_data_unc.csv'
train_unc_path = '/home/train_data_unc.csv'
cali_unc_path  = '/home/cali_data_unc.csv'

# Read the CSVs, setting 'time' as the index column
test_data      = pd.read_csv(test_path, index_col='time')
train_data     = pd.read_csv(train_path, index_col='time')
cali_data      = pd.read_csv(cali_path, index_col='time')

test_data_unc  = pd.read_csv(test_unc_path, index_col='time')
train_data_unc = pd.read_csv(train_unc_path, index_col='time')
cali_data_unc  = pd.read_csv(cali_unc_path, index_col='time')


# 2. Scale the Features with MinMaxScaler
#    and Keep as DataFrames
# -------------------------------
scaler = MinMaxScaler()

train_scaled     = pd.DataFrame(scaler.fit_transform(train_data),      index=train_data.index,      columns=train_data.columns)
test_scaled      = pd.DataFrame(scaler.transform(test_data),           index=test_data.index,       columns=test_data.columns)
cali_scaled      = pd.DataFrame(scaler.transform(cali_data),           index=cali_data.index,       columns=cali_data.columns)

train_unc_scaled = pd.DataFrame(scaler.fit_transform(train_data_unc),  index=train_data_unc.index,  columns=train_data_unc.columns)
test_unc_scaled  = pd.DataFrame(scaler.transform(test_data_unc),       index=test_data_unc.index,    columns=test_data_unc.columns)
cali_unc_scaled  = pd.DataFrame(scaler.transform(cali_data_unc),       index=cali_data_unc.index,    columns=cali_data_unc.columns)


# 3. Determine Optimal Number of Clusters via Elbow and Silhouette
# -------------------------------
# Compute correlation matrix on the training set
corr_mat = train_scaled.corr()

# 3a. Elbow Method: compute SSE for cluster counts 1–9
sse = []
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=2000, random_state=42)
    kmeans.fit(corr_mat)
    sse.append(kmeans.inertia_)

# Plot the elbow curve
frame_elbow = pd.DataFrame({'Cluster': range(1, 10), 'SSE': sse})
plt.figure(figsize=(10, 5))
plt.plot(frame_elbow['Cluster'], frame_elbow['SSE'], marker='o')
plt.xlabel('Number of clusters (k)')
plt.ylabel('SSE (Inertia)')
plt.title('Elbow Method for Optimal k')
plt.grid(True)
plt.show()

# 3b. Silhouette Method: evaluate silhouette score for k = 2–6
silhouette_scores = {}
for k in range(2, 7):
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(corr_mat)
    score = silhouette_score(corr_mat, labels)
    silhouette_scores[k] = score
    print(f"Silhouette Score for {k} clusters: {score:.4f}")

# Choose the k with the highest silhouette score
optimal_clusters = max(silhouette_scores, key=silhouette_scores.get)
print(f"\nOptimal number of clusters based on silhouette score: {optimal_clusters}")

# 4. Cluster Features on the Correlation Matrix
# -------------------------------
kmeans_final = KMeans(n_clusters=optimal_clusters, random_state=42)
cluster_labels = kmeans_final.fit_predict(corr_mat)

def print_clusters(data_df, labels):
    cluster_dict = {}
    for idx, lbl in enumerate(labels):
        feature = data_df.columns[idx]
        cluster_dict.setdefault(lbl, []).append(feature)
    for lbl, feats in cluster_dict.items():
        print(f"Cluster {lbl}: {', '.join(feats)}")

print("\nFeature clusters on training data:")
print_clusters(train_scaled, cluster_labels)

def create_clustered_dataframes(df, labels):
    clustered = {}
    for lbl in np.unique(labels):
        cols_in_lbl = [col for col, lab in zip(df.columns, labels) if lab == lbl]
        clustered[lbl] = df[cols_in_lbl]
    return clustered

# Create clustered DataFrames for train, test, and cali (features only)
clustered_train_dfs = create_clustered_dataframes(train_scaled, cluster_labels)
clustered_test_dfs  = create_clustered_dataframes(test_scaled, cluster_labels)
clustered_cali_dfs  = create_clustered_dataframes(cali_scaled, cluster_labels)

# Also prepare clustered DataFrames for uncertainty data (same column structure)
clustered_train_unc_dfs = create_clustered_dataframes(train_unc_scaled, cluster_labels)
clustered_test_unc_dfs  = create_clustered_dataframes(test_unc_scaled, cluster_labels)
clustered_cali_unc_dfs  = create_clustered_dataframes(cali_unc_scaled, cluster_labels)

# 5. Create Sequences for Each Cluster
# -------------------------------
def create_sequences(dataframes_dict, seq_length):
    sequences = {}
    for lbl, df in dataframes_dict.items():
        arr = df.values  # shape: (time_steps, n_features)
        xs = []
        for i in range(len(arr) - seq_length):
            xs.append(arr[i: i + seq_length])
        sequences[lbl] = np.stack(xs)  # shape: (n_samples, seq_length, n_features)
    return sequences

# Sequence length
T = 31  

# Create sequences for inputs
train_sequences     = create_sequences(clustered_train_dfs, T)
test_sequences      = create_sequences(clustered_test_dfs, T)
cali_sequences      = create_sequences(clustered_cali_dfs, T)

# Create sequences for uncertainties (same T) 
train_unc_sequences = create_sequences(clustered_train_unc_dfs, T)
#test_unc_sequences  = create_sequences(clustered_test_unc_dfs, T)
#cali_unc_sequences  = create_sequences(clustered_cali_unc_dfs, T)

# -------------------------------
# 6. Prepare Uncertainty Arrays
# -------------------------------
'''''
def stack_uncertainty(sequences_dict):
    """
    Given a dict of sequences for each cluster, stack them to form a single 3D array:
      - Step 1: collect in a list sorted by cluster label
      - Step 2: convert to numpy array of shape (n_clusters, n_samples, seq_length, n_features_per_cluster)
      - Step 3: rearrange to (n_samples, seq_length, total_n_clusters)
    """
    # List of arrays in ascending order of cluster labels
    arr_list = [sequences_dict[lbl] for lbl in sorted(sequences_dict.keys())]
    # Convert to array: shape (n_clusters, n_samples, seq_length, n_feats_cluster)
    arr_stack = np.stack(arr_list)
    # Now we want per-time-step, per-sample uncertainties across clusters:
    # Current shape: (n_clusters, n_samples, seq_length, n_feats_cluster)
    # But original code treats each cluster's full set as a single dimension. To match that:
    #   - First reshape each cluster to (n_samples, seq_length, 1) by collapsing its features
    #   - Then stack along last axis.
    # Instead, we will compute a single uncertainty per cluster per time step by averaging across features within that cluster:
    cluster_counts = [sequences_dict[lbl].shape[-1] for lbl in sorted(sequences_dict.keys())]
    n_clusters = len(arr_list)
    n_samples = arr_list[0].shape[0]
    # Create an empty array with shape (n_samples, seq_length, n_clusters)
    stacked = np.zeros((n_samples, T, n_clusters))
    for idx, lbl in enumerate(sorted(sequences_dict.keys())):
        # Mean across feature dimension within each cluster
        # arr_list[idx] has shape (n_samples, seq_length, n_feats_cluster)
        stacked[:, :, idx] = arr_list[idx].mean(axis=-1)
    return stacked  # shape: (n_samples, seq_length, n_clusters)

train_unc_array = stack_uncertainty(train_unc_sequences)
test_unc_array  = stack_uncertainty(test_unc_sequences)
cali_unc_array  = stack_uncertainty(cali_unc_sequences)
'''

train_unc_array = [train_unc_sequences [key] for key in sorted(train_unc_sequences .keys())]
train_unc_array2 =  np.array(train_unc_array)
train_unc_array = train_unc_array2.reshape(train_unc_array2.shape[1],train_unc_array2.shape[2],train_unc_array2.shape[0])
print("Uncertainty array shape:", train_unc_array.shape)


# 7. Define the VAE with Monte Carlo Dropout
# -------------------------------
class MCDropout(tf.keras.layers.Dropout):
    """Monte Carlo Dropout that remains active at inference time."""
    def call(self, inputs, training=None):
        return super().call(inputs, training=training or K.learning_phase())

def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim   = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def vae_model(input_sequences_list, kl_weight=0.75, dropout_rate=0.2):
    inputs   = []
    encoders = []
    # Sum of all features across clusters for the output dimension
    total_features = sum([X.shape[2] for X in input_sequences_list])

    # Build separate encoder branches for each cluster input
    for X in input_sequences_list:
        inp = Input(shape=(X.shape[1], X.shape[2]))  # (seq_length, n_features_cluster)
        h = LSTM(64, activation='relu', return_sequences=True)(inp)
        h = MCDropout(dropout_rate)(h)
        h = LSTM(32, activation='relu', return_sequences=True)(h)
        h = MCDropout(dropout_rate * 0.2)(h)
        h = LSTM(16, activation='relu', return_sequences=False)(h)
        z_mean   = Dense(16)(h)
        z_log_var= Dense(16)(h)
        z        = Lambda(sampling, output_shape=(16,))([z_mean, z_log_var])
        inputs.append(inp)
        encoders.append(z)

    # Concatenate latent vectors from all clusters
    concat = Concatenate(axis=-1)(encoders)
    # Decoder: repeat the concatenated latent vector for each time step
    dec = RepeatVector(input_sequences_list[0].shape[1])(concat)
    dec = LSTM(16, activation='relu', return_sequences=True)(dec)
    dec = LSTM(32, activation='relu', return_sequences=True)(dec)
    dec = LSTM(64, activation='relu', return_sequences=True)(dec)
    output = TimeDistributed(Dense(total_features))(dec)

    vae = Model(inputs=inputs, outputs=output)

    # Add KL divergence loss
    kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    vae.add_loss(kl_weight * K.mean(kl_loss))

    # Custom weighted MSE loss: accounts for per-sample uncertainty
    def weighted_mse_loss():
        def loss(y_true, y_pred):
            true_target     = y_true[:, :, : total_features]
            sample_uncer    = y_true[:, :, total_features:]
            weights         = 1.0 / (1.0 + sample_uncer)
            mse             = K.square(y_pred - true_target)
            weighted_mse    = mse * weights
            return K.mean(weighted_mse)
        return loss

    # Learning rate schedule
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=5e-4,
        decay_steps=1000,
        decay_rate=0.85
    )
    vae.compile(optimizer=Adam(learning_rate=lr_schedule), loss=weighted_mse_loss())
    return vae

def mc_dropout_predictions(model, X_list, n_simulations=100):
    """
    Perform multiple stochastic forward passes (Monte Carlo Dropout) to estimate mean
    and standard deviation of predictions.
    """
    preds = np.array([model.predict(X_list, batch_size=256, verbose=0) for _ in range(n_simulations)])
    mean_pred = preds.mean(axis=0)
    std_pred  = preds.std(axis=0)
    return mean_pred, std_pred

# 8. Prepare Data for Model Training
# -------------------------------
train_input_list = [train_sequences[lbl] for lbl in sorted(train_sequences.keys())]
test_input_list  = [test_sequences[lbl]  for lbl in sorted(test_sequences.keys())]
cali_input_list  = [cali_sequences[lbl]  for lbl in sorted(cali_sequences.keys())]

for idx, X in enumerate(train_input_list):
    print(f"Cluster {idx} train input shape: {X.shape}  # (n_samples, seq_length, n_feats)")

# Build the VAE model
vae = vae_model(train_input_list, kl_weight=0.75, dropout_rate=0.2)

true_data = np.concatenate(train_input_list, axis=-1)  # shape: (n_samples, seq_length, total_features)
unc_data  = train_unc_array.copy  

combined_target = np.concatenate([true_data, unc_data], axis=-1)
print(f"Combined training target shape: {combined_target.shape}")

# -------------------------------
# 9. Train the VAE
# -------------------------------
early_stop = EarlyStopping(
    monitor='val_loss',
    min_delta=1e-3,
    patience=10,
    verbose=1,
    mode='auto',
    restore_best_weights=True
)

history = vae.fit(
    train_input_list,
    combined_target,
    epochs=100,
    batch_size=128,
    validation_split=0.2,
    callbacks=[early_stop],
    shuffle=False
)

# 10. Evaluate on Test Set with MC Dropout
# -------------------------------
mean_pred_test, std_pred_test = mc_dropout_predictions(vae, test_input_list, n_simulations=100)

def sequences_to_original(X_seq, seq_length):
    n_samples, L, n_feats = X_seq.shape
    reconstructed = np.zeros((n_samples + seq_length, n_feats))
    for i in range(n_samples):
        reconstructed[i] = X_seq[i, 0]
    reconstructed[n_samples:] = X_seq[-1, :, :]
    return reconstructed

test_pred_recon        = sequences_to_original(mean_pred_test, T)
test_unc_recon         = sequences_to_original(std_pred_test, T)

def reconstruct_actual(clustered_dfs_dict):
    dfs = [clustered_dfs_dict[lbl] for lbl in sorted(clustered_dfs_dict.keys())]
    concat_df = pd.concat(dfs, axis=1)
    return concat_df.values

actual_test_values = reconstruct_actual(clustered_test_dfs)  
n_samples_test = mean_pred_test.shape[0]
mse_per_sample_test = np.mean((test_pred_recon[:n_samples_test] - actual_test_values[:n_samples_test])**2, axis=1)
uncertainty_per_sample_test = np.mean(test_unc_recon[:n_samples_test], axis=1)


results_test_df = pd.DataFrame({
    'mse_per_sample': mse_per_sample_test,
    'pred_uncertainty': uncertainty_per_sample_test
})
results_test_df.to_csv('/home/test_results.csv', index=False)

# Save the trained VAE model
vae.save('/home/model')

# 11. Calibrate on Calibration Set with MC Dropout
# -------------------------------
mean_pred_cali, std_pred_cali = mc_dropout_predictions(vae, cali_input_list, n_simulations=50)
cali_pred_recon = sequences_to_original(mean_pred_cali, T)
cali_unc_recon  = sequences_to_original(std_pred_cali, T)

actual_cali_values = reconstruct_actual(clustered_cali_dfs)

n_samples_cali = mean_pred_cali.shape[0]
mse_per_sample_cali = np.mean((cali_pred_recon[:n_samples_cali] - actual_cali_values[:n_samples_cali])**2, axis=1)
uncertainty_per_sample_cali = np.mean(cali_unc_recon[:n_samples_cali], axis=1)

results_cali_df = pd.DataFrame({
    'mse_per_sample': mse_per_sample_cali,
    'pred_uncertainty': uncertainty_per_sample_cali
})
results_cali_df.to_csv('/home/cali_results.csv', index=False)

print("\Completed. Results saved.")


In [ ]:
import numpy as np
from scipy import stats
import pandas as pd

def detect_pot_gdp(
    df: pd.DataFrame,
    column: str = 'run_1',
    threshold_pct: float = 95.0,
    anomaly_p: float = 0.5,
    output_col: str = 'pot_gdp'
) -> pd.DataFrame:
    df2 = df.copy()
    data = df2[column].dropna().values
    thresh = np.percentile(data, threshold_pct)
    peaks = data[data > thresh]
    shape, loc, scale = stats.genpareto.fit(peaks)
    all_vals = df2[column].values
    tail_p = 1.0 - stats.genpareto.cdf(all_vals, shape, loc, scale)
    df2[output_col] = (tail_p < anomaly_p).astype(int)
    # 8) (optional) store metadata in attrs
    df2[output_col].attrs = {
        'gpd_threshold':    float(thresh),
        'gpd_shape':        float(shape),
        'gpd_loc':          float(loc),
        'gpd_scale':        float(scale),
        'anomaly_p_level':  float(anomaly_p)
    }
    return df2

era = detect_pot_gdp(era, column='run_1', threshold_pct=95, anomaly_p=0.5, output_col='predicted_anomaly')
mar = detect_pot_gdp(mar, column='run_1', threshold_pct=95, anomaly_p=0.5, output_col='predicted_anomaly')
gemb = detect_pot_gdp(gemb, column='run_1', threshold_pct=95, anomaly_p=0.5, output_col='predicted_anomaly')